# Ch `11`: Concept `01`

## Multi RNN

All we need is TensorFlow:

In [1]:
import tensorflow as tf

First, define the constants. 

Let's say we're dealing with 1-dimensional vectors, and a maximum sequence size of 3.

In [2]:
input_dim = 1
seq_size = 3

Next up, define the placeholder(s). 

We only need one for this simple example: the input placeholder.

创建placeholder

In [3]:
input_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, seq_size, input_dim])

Now let's make a helper function to create LSTM cells

构造Lstm cell

In [4]:
def make_cell(state_dim):
    return tf.contrib.rnn.LSTMCell(state_dim)

Call the function and extract the cell outputs.

1、 LSTM堆积在一起，构造多层LSTM

In [5]:
with tf.variable_scope("first_cell") as scope:
    cell = make_cell(state_dim=10)
    outputs, states = tf.nn.dynamic_rnn(cell, input_placeholder, dtype=tf.float32)

You know what? We can just keep stacking cells on top of each other. In a new variable scope, you can pipe the output of the previous cell to the input of the new cell. Check it out:

In [6]:
with tf.variable_scope("second_cell") as scope:
    cell2 = make_cell(state_dim=10)
    outputs2, states2 = tf.nn.dynamic_rnn(cell2, outputs, dtype=tf.float32)

What if we wanted 5 layers of RNNs? 

There's a useful shortcut that the TensorFlow library supplies, called `MultiRNNCell`. Here's a helper function to use it:

2、 一次性构造多层LSTM

In [8]:
def make_multi_cell(state_dim, num_layers):
    cells = [make_cell(state_dim) for _ in range(num_layers)]
    return tf.contrib.rnn.MultiRNNCell(cells)

Here's the helper function in action:

In [9]:
multi_cell = make_multi_cell(state_dim=10, num_layers=5)
outputs5, states5 = tf.nn.dynamic_rnn(multi_cell, input_placeholder, dtype=tf.float32)

Before starting a session, let's prepare some simple input to the network.

训练模型

In [10]:
input_seq = [[1], [2], [3]]

Start the session, and initialize variables.

In [11]:
init_op = tf.global_variables_initializer()

sess = tf.InteractiveSession()
sess.run(init_op)

We can run the outputs to verify that the code is sound.

In [12]:
outputs_val, outputs2_val, outputs5_val = sess.run([outputs, outputs2, outputs5], 
                                                   feed_dict={input_placeholder: [input_seq]})
print(outputs_val)
print(outputs2_val)
print(outputs5_val)

[[[ 0.04565362 -0.02111018  0.00943931 -0.02890949  0.02353027 -0.07796901
    0.04569415 -0.00422793 -0.05205594  0.02865916]
  [ 0.10853661 -0.06479551  0.0178942  -0.07640032  0.05208643 -0.19509149
    0.08690087 -0.01186553 -0.14576697  0.0796235 ]
  [ 0.16930018 -0.12774967  0.02357387 -0.13736473  0.07771346 -0.30416703
    0.1011389  -0.01961475 -0.26182392  0.13400772]]]
[[[ 0.00877748  0.0036222   0.00051987 -0.01005535  0.00381236 -0.00712926
   -0.00404523 -0.01210839  0.01179848 -0.0004949 ]
  [ 0.02725317  0.01264807  0.00164001 -0.03059983  0.01529867 -0.02063793
   -0.01708099 -0.03790737  0.03866576 -0.00018147]
  [ 0.05122671  0.02603612  0.00436753 -0.05657636  0.03609543 -0.03538236
   -0.04079114 -0.0710327   0.07716534  0.00217945]]]
[[[ -4.57763645e-06   2.07890225e-05  -8.62103116e-06  -8.62272191e-06
     7.91534603e-07  -4.02755832e-05  -1.34765296e-05   2.37038912e-05
    -3.26332702e-05   2.98559316e-05]
  [ -2.76260434e-05   1.18018586e-04  -5.95373604e-05 